In [6]:
_PROMPT_TEMPLATE = """
请你提取下面段落的关键词，要求所提取的关键词能够鲜明表示内容性质。所提取的关键词应不少于5个。

待处理内容：

本次股票发行后拟在创业板市场上市，该市场具有较高的投资风险。创业
板公司具有业绩不稳定、经营风险高、退市风险大等特点，投资者面临较大的
市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风险因
素，审慎作出投资决定。
华瑞电器股份有限公司
Huarui Electrical Appliance Co., Ltd.
册地址：宁波市鄞州区姜山镇科技园区
首次公开发行股票并在创业板上市
招股意向书
发行人及全体董事、监事、高级管理人员承诺招股意向书不存在虚假记载、
误导性陈述或重大遗漏，并对其真实性、准确性、完整性、及时性承担个别和连
带的法律责任。
发行人及全体董事、监事、高级管理人员、发行人的控股股东、实际控制人
以及保荐人、承销的证券公司承诺因发行人招股意向书及其他信息披露资料有虚
假记载、误导性陈述或者重大遗漏，致使投资者在证券发行和交易中遭受损失的，
将依法赔偿投资者损失。

注意：你的回答不需要分段。你的回答格式应该按照下面的内容，请注意---output 等标记都必须输出，这是我用来提取答案的标记。
除了你的回答之外不要输出多余的文字。

---output
${{"keywords":[你提取的关键词]}}
"""

In [53]:
import dashscope
from dashscope import Generation
from http import HTTPStatus
import random
from CustomLLM import QWENonlie
from Container import g_container
apikey='sk-02b8b1a9175a4cae9bf9571d517d8b81'
llm=QWENonlie.from_name(model_name=Generation.Models.qwen_max,api_key=apikey)
g_container.MODEL = llm

In [9]:
llm._call(prompt=_PROMPT_TEMPLATE,
          stop=['---stop'])

'---output\n{"keywords":["创业板","投资风险","市场风险","信息披露","法律责任"]}'

In [54]:
%load_ext autoreload
%autoreload 2
from config import QWEN_CONFIG
from dashscope import TextEmbedding,BatchTextEmbedding
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import DashScopeEmbeddings,HuggingFaceEmbeddings

emb2=HuggingFaceEmbeddings(model_name='../../model/m3e/',encode_kwargs={'normalize_embeddings':True})
g_container.EMBEDDING=emb2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
from config import QWEN_CONFIG
cfg = QWEN_CONFIG(api_key=apikey,
                  embedding_name='text_embedding_async_v1')

In [ ]:
%load_ext autoreload
%autoreload 2
from TxtProcess import load_dir_txt_idx_process
cfg = QWEN_CONFIG(api_key='apikey',
                  embedding_name='text_embedding_async_v1')
kb=load_dir_txt_idx_process(txt_dir='../../bs_challenge_financial_14b_dataset/pdf_txt_file/',
                     pdf_dir='../../bs_challenge_financial_14b_dataset/pdf/',
                     cfg=cfg
                     )


In [60]:
from TxtProcess import faiss_kb_ds
kbidx=faiss_kb_ds(vs_path=cfg.vec_store_path,
                  kb_path='./',
                  embs=g_container.EMBEDDING)
kbidx.do_load(index='index')
kbidx.do_view()

32

In [31]:
%load_ext autoreload
%autoreload 2
from Tools.konwlegeqa import run_knowledge_qa

run_knowledge_qa(query='武汉兴图新科电子股份有限公司计划使用本次发行募集资金的多少用于补充流动资金？',
                 cfg=cfg)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'inputs': '武汉兴图新科电子股份有限公司计划使用本次发行募集资金的多少用于补充流动资金？',
 'outputs': '77.16%',
 'relent_docs': ['a0d278016e9baded15e0f5b3964563e525b1b787']}

In [52]:
import pdfplumber 
from prettytable import PrettyTable
with pdfplumber.open('../../bs_challenge_financial_14b_dataset/pdf/ba561001c2abd0714c413e2f6a77ffee58c644a6.PDF') as f:
    tables=f.pages[51].extract_tables(table_settings={"horizontal_strategy":"lines"})
    for table in tables:
        t=PrettyTable([str(i) for i in range(len(table[0]))])
        t.add_rows(table)
t.get_formatted_string()

'+-------+------+------+--------+----------+------+------------+--------------+------+---------------+------+\n|   0   |  1   |  2   |   3    |    4     |  5   |     6      |      7       |  8   |       9       |  10  |\n+-------+------+------+--------+----------+------+------------+--------------+------+---------------+------+\n|       | 序号 |      |        | 股东名称 |      |            | 持股数（股） |      | 持股比例（%） |      |\n|   1   | None | None | 牟金香 |   None   | None | 59,556,600 |     None     | None |     61.50     | None |\n|   2   | None | None | 陈建郎 |   None   | None | 9,684,000  |     None     | None |     10.00     | None |\n|   3   | None | None | 张有志 |   None   | None | 6,100,920  |     None     | None |      6.30     | None |\n|   4   | None | None | 王江涛 |   None   | None | 4,357,800  |     None     | None |      4.50     | None |\n|   5   | None | None | 东志刚 |   None   | None | 4,357,800  |     None     | None |      4.50     | None |\n|   6   | None | None | 张贤桂 |   None   | N